# Bookings held longer on detainer than on other charges

Here we're looking for bookings where the charge release date for immigration hold was greater than release date for other charges. Note that charge release dates may be later than actual release dates. What we're looking for is records where actual release date and hold release date are both later than release dates for other charges. Could consider adding additional logic below to control for this but for now we'll hand check.

In [7]:
import pandas as pd
import numpy as np

In [8]:
path = '../input/'
df = pd.read_csv(f'{path}pierce.csv.gz', sep='|', compression='gzip', low_memory=False)

In [9]:
df['booking_dt'] = pd.to_datetime(df['booking_dt'])
df['release_dt'] = pd.to_datetime(df['release_dt'])
df['charge_release_dt'] = pd.to_datetime(df['charge_release_dt'])

In [10]:
hold_ids = df[df['imm_hold'] == 1]['booking_id_hash'].tolist()

In [11]:
df['release_dt']= pd.to_datetime(df['release_dt'])
df['charge_release_dt'] = pd.to_datetime(df['charge_release_dt'])

df['charge_delta'] = df['charge_release_dt'] - df['booking_dt']
df['charge_delta'] = df['charge_delta'] / np.timedelta64(1,'h') 

df['charge_release_delta'] = df['charge_release_dt'] - df['release_dt']
df['charge_release_delta'] = df['charge_release_delta'] / np.timedelta64(1,'h')

held_longer_on_detainer = []
how_much_longer = []
for b_id in hold_ids:
    booking = df[df['booking_id_hash'] == b_id]

    non_hold_charge_deltas = booking[booking['booking_charge_desc'] != 'HOLD IMMIGRAT']['charge_delta']

    hold_charge_delta = float(booking[booking['booking_charge_desc'] == 'HOLD IMMIGRAT']['charge_delta'])
    
    hold_release_delta = float(booking[booking['booking_charge_desc'] == 'HOLD IMMIGRAT']['charge_release_delta'])
    
    if hold_charge_delta > non_hold_charge_deltas.max():
        
        held_longer_on_detainer.append(b_id)
    
    how_much_longer.append([hold_charge_delta - non_hold_charge_deltas.max(), hold_release_delta, b_id])

# df[df['booking_id_hash'].isin(held_longer_on_detainer)].to_csv('../output/held_longer_on_detainer.csv', index=False)

In [12]:
held_longer_on_detainer_df = df[df['booking_id_hash'].isin(held_longer_on_detainer)]

In [13]:
len(set(held_longer_on_detainer_df['booking_id_hash']))

45

In [14]:
temp = pd.DataFrame(how_much_longer, columns=['longer_than_other_charge', 'longer_than_release', 'booking_id_hash']).drop_duplicates()

In [15]:
temp['longer_than_other_charge'].mean()

12.424556737588654

In [16]:
temp['longer_than_release'].mean()

-10.518794326241137

In [39]:
temp

,longer_than_other_charge,longer_than_release,booking_id_hash
0,9.000000,3.000000,107dfc6e2baea3003a41ebf3d9d3ff3479e39ac6
8,-1.000000,-1.000000,1e888d705dc6a92efacdc4acc7a0b1ba0e83f9b4
13,0.000000,-0.750000,f6bb19f15053bb16348817cc70c0019cb660e63a
18,0.000000,0.000000,947bb503f92ea304e8d4c7704d6babb916b7f41b
23,2.500000,-0.166667,5910dc7c2845af13894b7a836d1c17daf5c27199
30,0.000000,-2.000000,f40892956eafcc693743c927af5975e9ee576dc6
38,0.000000,0.000000,e4cf80ead043b7110d96853c13eacc6d8fe89ec7
41,-0.500000,0.000000,106f0483a7fc263de773278d41372adbf56e26d4
46,0.000000,0.000000,da535f7c1916265ddfe3830c971227cbfa45c045
49,0.000000,0.000000,c9714691c574b266c2e22194cfecd573be780a47


In [17]:
hold_df = df[df['imm_hold'] == 1]

In [18]:
hold_df = pd.merge(hold_df, temp, on='booking_id_hash')

In [19]:
hold_df[['booking_dt', 'release_dt', 'booking_charge_desc', 'charge_release_dt', 'cause_num', 'release_disposition_desc', 'gender_desc', 'race_desc']].head()

,booking_dt,release_dt,booking_charge_desc,charge_release_dt,cause_num,release_disposition_desc,gender_desc,race_desc
0,2015-12-22 08:21:00,2018-04-19 14:00:00,THFT OF FIREARM (B),2018-04-18 12:00:00,12-1-00068-1,Dismissed,MALE,ASIAN/PACIFIC ISLAND
1,2015-12-22 08:21:00,2018-04-19 14:00:00,TRAFFICKING STOL PROP 1,2018-04-18 12:00:00,12-1-00068-1,Dismissed,MALE,ASIAN/PACIFIC ISLAND
2,2015-12-22 08:21:00,2018-04-19 14:00:00,TRAFFICKING STOL PROP 1,2018-04-18 12:00:00,12-1-00068-1,Dismissed,MALE,ASIAN/PACIFIC ISLAND
3,2015-12-22 08:21:00,2018-04-19 14:00:00,HOLD IMMIGRAT,2018-04-19 17:00:00,NaN,Immigration,MALE,ASIAN/PACIFIC ISLAND
4,2015-12-22 08:21:00,2018-04-19 14:00:00,HOLD WSI,2018-04-19 08:00:00,NaN,Cancel Detainer,MALE,ASIAN/PACIFIC ISLAND


In [20]:
hold_df.drop_duplicates('booking_id_hash', inplace=True)

In [21]:
hold_df['longer_than_release'].mean()

-10.518794326241137

In [22]:
hold_df['longer_than_other_charge'].mean()

12.424556737588654

In [25]:
hold_df['longer_than_other_charge'].mean()

12.424556737588654

In [37]:
hold_df[hold_df['longer_than_other_charge'] > 24]['longer_than_other_charge']

89       76.000000
429     993.000000
440    1416.000000
449      89.433333
478    1631.000000
524     225.000000
574      69.000000
Name: longer_than_other_charge, dtype: float64

In [35]:
hold_df[hold_df['longer_than_other_charge'] > 0]

,booking_dt,release_dt,booking_charge_desc,booking_charge_txt,charge_release_dt,title_txt,cause_num,jurisdiction_cd,release_disposition_desc,post_flg,...,violent,felony,misdemeanor,total_fine,paid_bail,log_total_fine,charge_delta,charge_release_delta,longer_than_other_charge,longer_than_release
0,2015-12-22 08:21:00,2018-04-19 14:00:00,THFT OF FIREARM (B),NaN,2018-04-18 12:00:00,"Report Criteria: 10/01/2017 to 07/31/2018, Dis...",12-1-00068-1,SUP,Dismissed,N,...,0,1,0,0.0,False,NaN,20355.650000,-26.000000,9.000000,3.000000
23,2016-05-29 22:14:00,2017-07-09 10:40:00,HIT/RUN UNATT,NaN,2016-07-07 08:00:00,"Report Criteria: 10/01/2016 to 09/30/2017, Dis...",B250561,TACM,Time Served,N,...,0,1,0,0.0,False,NaN,921.766667,-8810.666667,2.500000,-0.166667
52,2016-08-06 21:06:00,2017-05-03 16:43:00,ASLT 1,NaN,2017-05-03 11:59:00,"Report Criteria: 10/01/2016 to 09/30/2017, Dis...",16-1-03191-1,SUP,Credit for Time Served,N,...,1,1,0,0.0,False,NaN,6470.883333,-4.733333,4.516667,-0.216667
57,2016-09-06 16:46:00,2016-10-20 13:05:00,HOLD IMMIGRAT,NaN,2016-10-20 13:05:00,"Report Criteria: 10/01/2016 to 09/30/2017, Dis...",NaN,IMM,Immigration,N,...,0,1,0,0.0,False,NaN,1052.316667,0.000000,1.150000,0.000000
84,2016-09-25 01:35:00,2016-12-15 08:45:00,HOLD IMMIGRAT,NaN,2016-12-15 08:45:00,"Report Criteria: 10/01/2016 to 09/30/2017, Dis...",NaN,IMM,Immigration,N,...,0,1,0,0.0,False,NaN,1951.166667,0.000000,0.750000,0.000000
89,2016-10-03 14:22:00,2016-11-15 09:05:00,HOLD IMMIGRAT,NaN,2016-11-15 12:00:00,"Report Criteria: 10/01/2016 to 09/30/2017, Dis...",NaN,IMM,Immigration,N,...,0,0,1,0.0,False,NaN,1029.633333,2.916667,76.000000,2.916667
94,2016-10-24 00:17:00,2016-11-04 23:48:00,ASSAULT 4 /DV,NaN,2016-11-04 08:00:00,"Report Criteria: 10/01/2016 to 09/30/2017, Dis...",6ZC002709,PD1,Time Served,N,...,1,1,0,0.0,False,NaN,271.716667,-15.800000,15.983333,0.183333
133,2017-01-12 20:24:00,2017-07-12 09:30:00,RECK ENDANGER,NaN,2017-05-16 17:00:00,"Report Criteria: 10/01/2016 to 09/30/2017, Dis...",17-1-00164-5,SUP,Dismissed,N,...,1,1,0,0.0,False,NaN,2972.600000,-1360.500000,2.000000,0.500000
138,2017-01-14 04:32:00,2018-04-04 14:25:00,RAPE 2,ATTEMPTED,2018-04-04 14:00:00,"Report Criteria: 10/01/2017 to 07/31/2018, Dis...",17-1-00189-1,SUP,Dismissed,N,...,0,1,0,0.0,False,NaN,10689.466667,-0.416667,0.416667,0.000000
146,2017-01-25 16:16:00,2017-05-02 17:31:00,POSSESS STOLEN VEHICLE,NaN,2017-05-02 17:00:00,"Report Criteria: 10/01/2016 to 09/30/2017, Dis...",16-1-02424-8,SUP,Credit for Time Served,N,...,0,1,0,0.0,False,NaN,2328.733333,-0.516667,3.000000,2.483333


In [38]:
hold_df.loc[478]

booking_dt                                                2017-10-02 12:44:00
release_dt                                                2018-01-12 07:00:00
booking_charge_desc                                             HOLD IMMIGRAT
booking_charge_txt                                                        NaN
charge_release_dt                                         2018-01-12 07:00:00
title_txt                   Report Criteria: 10/01/2017 to 07/31/2018, Dis...
cause_num                                                                 NaN
jurisdiction_cd                                                           IMM
release_disposition_desc                                          Extradition
post_flg                                                                    N
booking_id_hash                      1fba618ef09195b518348977fbd5a811222f6841
gender_desc                                                              MALE
race_desc                                                       